In [ ]:
"""
Single galaxy simulations: determine the shear dependence on PSF higher moment errors.
"""

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import galsim
from IPython.display import clear_output
from astropy.io import fits
from matplotlib.colors import LogNorm
from numpy import mgrid, sum
import scipy.linalg as alg
import scipy.stats as stats
from galsim.zernike import Zernike
import matplotlib

In [ ]:
import sys
sys.path.append('../psfhome')

from homesm import *
from metasm import *
from moments import *
from HOMExShapeletPair import *

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 14

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def do_tests(tests,j,test_m, test_c,n):
    testsresult=[]
    for i in range(len(tests)):
        test = HOMExShapeletPair(*tests[i][:-1],**tests[i][-1])

        test.setup_shapelet_psf(test_m[i],test_c[i],n)

        results = test.get_results(metacal = False)
        testsresult.append(results)
        clear_output() 
        print ("Finished "+str(float((i+1))/len(tests)*100)+"%")
    return testsresult
    

In [ ]:
def do_tests_speed(tests,j,test_m, test_c,n):
    testsresult=[]
    for i in range(len(tests)):
        test = HOMExShapeletPair(*tests[i][:-1],**tests[i][-1])
        if i!=0:
            test.speed_setup_shapelet_psf(test_m[i],test_c[i],n,psf_light, psf_model_light, dm)
        else:
            test.setup_shapelet_psf(test_m[i],test_c[i],n)
            psf_light = test.psf_light
            psf_model_light = test.psf_model_light
            dm = test.dm
        results = test.get_results(metacal = False)
        testsresult.append(results)
        #clear_output() 
        #print ("Finished "+str(float((i+1))/len(tests)*100)+"%")
    return testsresult
    

In [ ]:
def e2(e1,e):
    return np.sqrt(e**2 - e1**2)

In [ ]:
test1 = HOMExShapeletPair("gaussian", 3.0, 0.2, 0.2, 0.01, 0.01, "gaussian", 2.0)

In [ ]:
m = np.zeros(12)
c = np.zeros(12)
c[9]+=0.001
test1.setup_shapelet_psf(m,c,4)
pqlist = test1.sxm.get_pq_full(6)[3:]


In [ ]:

test2_init = [("gaussian" ,0.85    ,0.28,0.28,1e-8,1e-8,"gaussian"  ,1.2,{'subtract_intersection':True}) for i in range(21)
]
test2_m = np.zeros(shape = (22,21,25))
test2_c = np.zeros(shape = (22,21,25))
for index in range(22):
    for i in range(21):
        test2_c[index][i][index+3]+=-0.01 + 0.001*i



In [ ]:
test2result = []
for i in range(len(test2_m)):
    print( "Start tests for moment"+ str(i+4))
    test2result.append(do_tests(test2_init,i,test2_m[i],test2_c[i],6))
    #print test2result
%store test2result

In [ ]:
pqlist = test1.sxm.get_pq_full(6)[3:]
fig = plt.figure(figsize = (21,12))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)

param1_dir = {}
param2_dir = {}


for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    ax = plt.subplot(4,7,1+7*(n-3)+p)
    
    dm = np.array([t['dm'][j+3] for t in test2result[j]])
    dg1 = np.array([t["abs_bias"][0] for t in test2result[j]])
    dg2 = np.array([t["abs_bias"][1] for t in test2result[j]])
    
    params1= np.polyfit(dm,dg1,2)
    params2= np.polyfit(dm,dg2,2)
    
#     print params1
    
    
    plt.plot(dm,dg1,label='g1')
    plt.plot(dm,dg2,label='g2')

    dg1_project = params1[2] + dm*params1[1] + dm**2*params1[0]
    dg2_project = params2[2] + dm*params2[1] + dm**2*params2[0]
    
    
    plt.plot(dm,dg1_project)
    plt.plot(dm,dg2_project)
    
    param1_dir[(p,q)] = params1
    param2_dir[(p,q)] = params2
    #print test4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    plt.ticklabel_format(axis='y',style='sci',scilimits=(0,3))

    plt.xlabel(r"$\Delta m_{p,q}$")
    plt.ylabel(r'$\Delta g_i$')
    
    
    
    plt.title(str((p,q)))
    #plt.show()
    plt.legend()
    
#fig.colorbar(axes)

In [ ]:
import pickle


with open("../plots2/pickle/shear_response.pkl","wb") as f:
    pickle.dump([pqlist,test2result],f)




In [ ]:
import pickle

with open("../plots2/pickle/shear_response.pkl","rb") as f:
    pqlist,test2result = pickle.load(f)


In [ ]:
import pickle

f = open("../notebook/data/params1.pkl","wb")
pickle.dump(param1_dir,f)
f.close()

f = open("../notebook/data/params2.pkl","wb")
pickle.dump(param2_dir,f)
f.close()



In [ ]:


HSC_moment_bias = np.load('data/mean_residual.npy')


In [ ]:
#gal_size = 0.17 arcsec, psf_size = 0.24 arcsec, pixel_size = 0.2 arcsec
test3_init = [("gaussian" ,0.85    ,0.28,0.28,0.001+0.001*i,0.001+0.001*i,"gaussian"  ,1.2 ,{'subtract_intersection':True}) for i in range(10)
]

# test3_init = [("gaussian" ,3.98    ,0.28,0.28,0.001+0.001*i,0.001+0.001*i,"gaussian"  ,2.4 ,{'subtract_intersection':True}) for i in range(10)
# ]
test3_m = np.zeros(shape = (22,10,25))
test3_c = np.zeros(shape = (22,10,25))
for index in range(22):
    for i in range(10):
        test3_c[index][i][index+3]+=HSC_moment_bias[index+3]
        #test3_c[index][i][index+3]+=0.005



In [ ]:
test3result = []
for i in range(len(test3_m)):
    print( "Start tests for moment"+ str(i+4))
    test3result.append(do_tests_speed(test3_init,i,test3_m[i],test3_c[i],6))
%store test3result
 

In [ ]:
import pickle


with open("../plots2/pickle/add_and_mul.pkl","wb") as f:
    pickle.dump([pqlist,test3result,test3_init],f)



In [ ]:
pqlist = test1.sxm.get_pq_full(6)[3:]
fig = plt.figure(figsize = (21,12))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)

g1_m = [];g1_c = [];g2_m = [];g2_c = []


for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    ax = plt.subplot(4,7,1+7*(n-3)+p)
    
    g1 = np.array([param[4] for param in test3_init])
    g2 = np.array([param[5] for param in test3_init])
    
    dg1 = np.array([t["abs_bias"][0] for t in test3result[j]])
    dg2 = np.array([t["abs_bias"][1] for t in test3result[j]])
    
    params1= np.polyfit(g1,dg1,1)
    params2= np.polyfit(g2,dg2,1)
    
    g1_m.append(params1[0]);g1_c.append(params1[1]);g2_m.append(params2[0]);g2_c.append(params2[1])
    
    #print params1,params2
    
    dg1_project = params1[1] + g1*params1[0] 
    dg2_project = params2[1] + g2*params2[0] 
    
    plt.plot(g1,dg1,'.',label='g1')
    plt.plot(g2,dg2,'.',label='g2')
    
    plt.plot(g1,dg1_project)
    plt.plot(g2,dg2_project)
    
    
    print(str((p,q)), (params1[0]/0.005, params2[0]/0.005), (params1[1]/0.005, params2[1]/0.005))
    
    plt.ticklabel_format(axis='y',style='sci',scilimits=(0,3))

    #print test4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    
    plt.xlabel(r"$g_1$")
    plt.ylabel(r'${\Delta g_i}$')
    
    plt.title(str((p,q)))
    #plt.show()
    plt.legend()
    
#fig.colorbar(axes)

In [ ]:




test4_init = [("gaussian" ,3.98    ,-0.2+i*0.04,e2(-0.2+i*0.04, 0.28),1e-8,1e-8,"gaussian"  ,2.4,{'subtract_intersection':True}) for i in range(5)
]
test4_m = np.zeros(shape = (22,5,25))
test4_c = np.zeros(shape = (22,5,25))
for index in range(22):
    for i in range(5):
        test4_c[index][i][index+3]+=0.005


In [ ]:
test4result = []
for i in range(len(test4_m)):
    print "Start tests for moment"+ str(i+4)
    test4result.append(do_tests(test4_init,i,test4_m[i],test4_c[i],6))
%store test4result
 

In [ ]:
print test4result[1]

In [ ]:
pqlist = test1.sxm.get_pq_full(6)[3:]
fig = plt.figure(figsize = (21,12))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)

param1_dir = {}
param2_dir = {}


for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    ax = plt.subplot(4,7,1+7*(n-3)+p)
    
    e1  = np.array([t['e1'] for t in test4result[j]])
    dg1 = np.array([t["abs_bias"][0] for t in test4result[j]])
    dg2 = np.array([t["abs_bias"][1] for t in test4result[j]])

    
    
    plt.plot(e1,dg1,label='g1')
    plt.plot(e1,dg2,label='g2')


    #print te bvst4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    plt.ticklabel_format(axis='y',style='sci',scilimits=(0,3))

    plt.xlabel(r"$ e_1$")
    plt.ylabel(r'$\Delta g_i$')
    
    
    
    plt.title(str((p,q)))
    #plt.show()
    plt.legend()
    
#fig.colorbar(axes)

$g(x) = (1+m_1(x)+m_2(x)+\dots)g_{true}(x)$

$<g(x) g(x+\theta)> = <((1+m_1(x)+m_2(x)+\dots)g_{true}(x))((1+m_1(x+\theta)+m_2(x+\theta)+\dots)g_{true}(x+\theta))>$
$= <g_{true}(x)g_{true}(x+\theta)> + <(m_1(x)+m_2(x)+\dots)g_{true}(x) g_{true}(x+\theta)> + <(m_1(x+\theta)+m_2(x+\theta)+\dots)g_{true}(x)g_{true}(x+\theta)>$
$= \xi_{true} + 2 \sum_i < m_i(x)> \xi_{true}$

In [ ]:
nob = 50

label_list = []
pqlist = test1.sxm.get_pq_full(6)
for i in range(nob):
    if i < 25:
        i_pre = 't'
        
    else:
        i_pre = 'r'
        
    
    label1 = i_pre+str(pqlist[i%25][0])+str(pqlist[i%25][1])
    label_list.append(label1)

fig, ax = plt.subplots(1,1,figsize=(8, 6)) 
ax.plot(np.arange(3,25),g1_m,'o',label = 'm1')
ax.plot(np.arange(3,25),g2_m,'o',label = 'm2')

ax.axvspan(7, 11, color='r', alpha=0.2, lw=0)

ax.axvspan(18, 24, color='r', alpha=0.2, lw=0)


ax.set_xticks(np.arange(3,25,1))
ax.set_xticklabels(label_list[28:], rotation='vertical', fontsize=14)

plt.grid()
plt.legend()
plt.ylabel("Multiplicative Bias")




print( "m1 = " + str(np.sum(g1_m)))
print( "m2 = " + str(np.sum(g2_m)))

In [ ]:

fig, ax = plt.subplots(1,1,figsize=(8, 6)) 
#ax.plot(np.arange(3,25),g1_c,'o',label = 'c1')
ax.plot(np.arange(3,25),g2_c,'o',label = 'c2')

ax.axvspan(7, 11, color='r', alpha=0.2, lw=0)

ax.axvspan(18, 24, color='r', alpha=0.2, lw=0)

ax.set_xticks(np.arange(3,25,1))
ax.set_xticklabels(label_list[28:], rotation='vertical', fontsize=14)

plt.grid()
plt.legend()
plt.ylabel("Additive Bias")
plt.yscale('symlog', linthresh = 0.00001)




print( "c1 = " + str(np.sum(g1_c)))
print( "c2 = " + str(np.sum(g2_c)))

In [ ]:
g = [param[4] for param in test3_init]
dg1 = np.array([t["abs_bias"][0] for t in test3result[4]])+2*np.array([t["abs_bias"][0] for t in test3result[6]])+np.array([t["abs_bias"][0] for t in test3result[8]])
dg2 = np.array([t["abs_bias"][1] for t in test3result[4]])+2*np.array([t["abs_bias"][1] for t in test3result[6]])+np.array([t["abs_bias"][1] for t in test3result[8]])


plt.plot(g,dg1)
plt.plot(g,dg2)




In [ ]:
#change coma1 
#d(trefoil1) = du^3 - 3 d(uv^2) = 0
#d(coma1) = 0.04


test51_init = [("gaussian" ,3.0    ,0.28,0.28,0.001+0.001*i,0.001+0.001*i,"gaussian"  ,2.0,{'subtract_intersection':True}) for i in range(10)
]
test51_m = np.zeros(shape = (10,12))
test51_c = np.zeros(shape = (10,12))
for i in range(test51_c.shape[0]):
    test51_c[i][6]+=0.03
    test51_c[i][4]+=0.01

test51result = []
for i in range(len(test51_m)):
    test = HOMExShapeletPair(*test51_init[i][:-1],**test51_init[i][-1])
    test.setup_shapelet_psf(test51_m[i],test51_c[i],4)
    results = test.get_results()
    test51result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test51_m)*100)+"%")


In [ ]:
#change coma2
#d(trefoil2) = 3d(u^2 v) - d(v^3) = 0
#d(coma1) = 0.04

test52_init = [("gaussian" ,3.0    ,0.28,0.28,0.001+0.001*i,0.001+0.001*i,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test52_m = np.zeros(shape = (10,12))
test52_c = np.zeros(shape = (10,12))
for i in range(test52_c.shape[0]):
    test52_c[i][5]+=0.01
    test52_c[i][3]+=0.03

test52result = []
for i in range(len(test52_m)):
    test = HOMExShapeletPair(*test52_init[i][:-1],**test52_init[i][-1])
    test.setup_shapelet_psf(test52_m[i],test52_c[i],4)
    results = test.get_results()
    test52result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test52_m)*100)+"%")


In [ ]:
#change trefoil1 
#d(coma1) = du^3 + d(uv^2) = 0
#d(coma1) = 0.04


test53_init = [("gaussian" ,3.0    ,0.28,0.28,0.001+0.001*i,0.001+0.001*i,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test53_m = np.zeros(shape = (10,12))
test53_c = np.zeros(shape = (10,12))
for i in range(test53_c.shape[0]):
    test53_c[i][6]+=0.01
    test53_c[i][4]-=0.01

test53result = []
for i in range(len(test53_m)):
    test = HOMExShapeletPair(*test53_init[i][:-1],**test53_init[i][-1])
    test.setup_shapelet_psf(test53_m[i],test53_c[i],4)
    results = test.get_results()
    test53result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test53_m)*100)+"%")


In [ ]:
#change trefoil2
#d(coma2) = d(u^2 v) - d(v^3) = 0
#d(coma1) = 0.04


test54_init = [("gaussian" ,3.0    ,0.28,0.28,0.001+0.001*i,0.001+0.001*i,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test54_m = np.zeros(shape = (10,12))
test54_c = np.zeros(shape = (10,12))
for i in range(test54_c.shape[0]):
    test54_c[i][5]+=0.01
    test54_c[i][3]-=0.01

test54result = []
for i in range(len(test54_m)):
    test = HOMExShapeletPair(*test54_init[i][:-1],**test54_init[i][-1])
    test.setup_shapelet_psf(test54_m[i],test54_c[i],4)
    results = test.get_results()
    test54result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test54_m)*100)+"%")


In [ ]:
plt.plot([param[4] for param in test51_init],np.array([t["abs_bias"][0] for t in test51result]),label='coma1', color = 'blue')
plt.plot([param[5] for param in test51_init],np.array([t["abs_bias"][1] for t in test51result]),'-.', color = 'blue')
plt.plot([param[4] for param in test52_init],np.array([t["abs_bias"][0] for t in test52result]),label='coma2', color = 'orange')
plt.plot([param[5] for param in test52_init],np.array([t["abs_bias"][1] for t in test52result]),'-.', color = 'orange')
plt.plot([param[4] for param in test53_init],np.array([t["abs_bias"][0] for t in test53result]),label='trefoil1', color = 'green')
plt.plot([param[5] for param in test53_init],np.array([t["abs_bias"][1] for t in test53result]),'-.',color = 'green')
plt.plot([param[4] for param in test54_init],np.array([t["abs_bias"][0] for t in test54result]),label='trefoil2',color = 'purple')
plt.plot([param[5] for param in test54_init],np.array([t["abs_bias"][1] for t in test54result]),'-.',color = 'purple')
plt.xlabel(r'$g$')
plt.ylabel(r'$\Delta g$')
plt.title('g1 solid, g2 dashed')
plt.legend()

In [ ]:
test61_init = [("gaussian" ,4.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0+0.5*i,{'subtract_intersection':True}) for i in range(20)
]
test61_m = np.zeros(shape = (20,12))
test61_c = np.zeros(shape = (20,12))
for i in range(test61_c.shape[0]):
    test61_c[i][6]+=0.03
    test61_c[i][4]+=0.01

test61result = []
for i in range(len(test61_m)):
    test = HOMExShapeletPair(*test61_init[i][:-1],**test61_init[i][-1])
    test.setup_shapelet_psf(test61_m[i],test61_c[i],4)
    results = test.get_results()
    test61result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test61_m)*100)+"%")


In [ ]:
test62_init = [("gaussian" ,4.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0+0.5*i,{'subtract_intersection':True}) for i in range(20)
]
test62_m = np.zeros(shape = (20,12))
test62_c = np.zeros(shape = (20,12))
for i in range(test62_c.shape[0]):
    test62_c[i][5]+=0.01
    test62_c[i][3]+=0.03

test62result = []
for i in range(len(test62_m)):
    test = HOMExShapeletPair(*test62_init[i][:-1],**test62_init[i][-1])
    test.setup_shapelet_psf(test62_m[i],test62_c[i],4)
    results = test.get_results()
    test62result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test62_m)*100)+"%")


In [ ]:
test63_init = [("gaussian" ,4.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0+0.5*i,{'subtract_intersection':True}) for i in range(20)
]
test63_m = np.zeros(shape = (20,12))
test63_c = np.zeros(shape = (20,12))
for i in range(test63_c.shape[0]):
    test63_c[i][6]+=0.01
    test63_c[i][4]-=0.01

test63result = []
for i in range(len(test63_m)):
    test = HOMExShapeletPair(*test63_init[i][:-1],**test63_init[i][-1])
    test.setup_shapelet_psf(test63_m[i],test63_c[i],4)
    results = test.get_results()
    test63result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test63_m)*100)+"%")


In [ ]:
test64_init = [("gaussian" ,4.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0+0.5*i,{'subtract_intersection':True}) for i in range(20)
]
test64_m = np.zeros(shape = (20,12))
test64_c = np.zeros(shape = (20,12))
for i in range(test64_c.shape[0]):
    test64_c[i][5]+=0.01
    test64_c[i][3]-=0.01

test64result = []
for i in range(len(test64_m)):
    test = HOMExShapeletPair(*test64_init[i][:-1],**test64_init[i][-1])
    test.setup_shapelet_psf(test64_m[i],test64_c[i],4)
    results = test.get_results()
    test64result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test64_m)*100)+"%")


In [ ]:
plt.figure(figsize = (8,6))

plt.plot([t['gal_sigma']/t['psf_sigma'] for t in test61result],np.array([ t["abs_bias"][0]/0.02 for t in test61result]),label = 'coma1',color = 'blue')
plt.plot([t['gal_sigma']/t['psf_sigma'] for t in test61result],np.array([ t["abs_bias"][1]/0.02 for t in test61result]),'-.',color = 'blue')

plt.plot([t['gal_sigma']/t['psf_sigma'] for t in test62result],np.array([ t["abs_bias"][0]/0.02 for t in test62result]),label = 'coma2',color = 'orange')
plt.plot([t['gal_sigma']/t['psf_sigma'] for t in test62result],np.array([ t["abs_bias"][1]/0.02 for t in test62result]),'-.',color = 'orange')

plt.plot([t['gal_sigma']/t['psf_sigma'] for t in test63result],np.array([ t["abs_bias"][0]/0.02 for t in test63result]),label = 'trefoil1',color = 'green')
plt.plot([t['gal_sigma']/t['psf_sigma'] for t in test63result],np.array([ t["abs_bias"][1]/0.02 for t in test63result]),'-.',color = 'green')

plt.plot([t['gal_sigma']/t['psf_sigma'] for t in test64result],np.array([ t["abs_bias"][0]/0.02 for t in test64result]),label = 'trefoil2',color = 'purple')
plt.plot([t['gal_sigma']/t['psf_sigma'] for t in test64result],np.array([ t["abs_bias"][1]/0.02 for t in test64result]),'-.',color = 'purple')

plt.xlabel(r"$\sigma_{galaxy}/\sigma_{PSF}$")
plt.ylabel(r'$\frac{\delta g}{\delta_{moment}}$')

plt.title('Gaussian galaxy & Gaussian PSF')

plt.legend()

In [ ]:
test71_init = ("gaussian" ,4.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,4.0,{'subtract_intersection':True}) 
test71_m = np.zeros(shape = (1,12))
test71_c = np.zeros(shape = (1,12))

test71_c[0][6]+=0.03
test71_c[0][4]+=0.03

test = HOMExShapeletPair(*test71_init[:-1],**test71_init[-1])
test.setup_shapelet_psf(test71_m[0],test71_c[0],4)
truth = test.psf_light
model = test.psf_model_light

residual = model.drawImage(scale = 1.0, nx = 100, ny = 100).array - truth.drawImage(scale = 1.0, nx = 100, ny = 100).array



In [ ]:

fig,ax = plt.subplots(figsize = (3,3))
ax.set_xticks([])
ax.set_yticks([])

plt.title('coma 1 residual')
plt.imshow(residual, vmin = -0.001, vmax = 0.001)
#plt.colorbar()
plt.show()

fig,ax = plt.subplots(figsize = (3,3))
ax.set_xticks([])
ax.set_yticks([])
plt.imshow(model.drawImage(scale = 1.0, nx = 100, ny = 100).array, cmap=plt.cm.BuPu)
plt.title('coma 1')
#plt.colorbar()
plt.show()



In [ ]:
test72_init = ("gaussian" ,4.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,4.0,{'subtract_intersection':True}) 
test72_m = np.zeros(shape = (1,12))
test72_c = np.zeros(shape = (1,12))

test72_c[0][3]+=0.03
test72_c[0][5]+=0.03

test = HOMExShapeletPair(*test72_init[:-1],**test72_init[-1])
test.setup_shapelet_psf(test72_m[0],test72_c[0],4)
truth = test.psf_light
model = test.psf_model_light

residual = model.drawImage(scale = 1.0, nx = 100, ny = 100).array - truth.drawImage(scale = 1.0, nx = 100, ny = 100).array



In [ ]:

fig,ax = plt.subplots(figsize = (3,3))
ax.set_xticks([])
ax.set_yticks([])

plt.imshow(residual, vmin = -0.001, vmax = 0.001)
plt.title('coma 2 residual')
plt.show()

fig,ax = plt.subplots(figsize = (3,3))
ax.set_xticks([])
ax.set_yticks([])

plt.imshow(model.drawImage(scale = 1.0, nx = 100, ny = 100).array, cmap=plt.cm.BuPu)
plt.title('coma 2')
#plt.colorbar()
plt.show()


In [ ]:
test73_init = ("gaussian" ,4.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,4.0,{'subtract_intersection':True}) 
test73_m = np.zeros(shape = (1,12))
test73_c = np.zeros(shape = (1,12))

test73_c[0][6]+=0.02
test73_c[0][4]-=0.06

test = HOMExShapeletPair(*test73_init[:-1],**test73_init[-1])
test.setup_shapelet_psf(test73_m[0],test73_c[0],4)
truth = test.psf_light
model = test.psf_model_light

residual = model.drawImage(scale = 1.0, nx = 100, ny = 100).array - truth.drawImage(scale = 1.0, nx = 100, ny = 100).array



In [ ]:

fig,ax = plt.subplots(figsize = (3,3))
ax.set_xticks([])
ax.set_yticks([])

plt.imshow(residual, vmin = -0.001, vmax = 0.001)
plt.title('trefoil 1 residual')
plt.show()

fig,ax = plt.subplots(figsize = (3,3))
ax.set_xticks([])
ax.set_yticks([])

plt.imshow(model.drawImage(scale = 1.0, nx = 100, ny = 100).array, cmap=plt.cm.BuPu)
plt.title('trefoil 2')
#plt.colorbar()
plt.show()


In [ ]:
test74_init = ("gaussian" ,4.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,4.0,{'subtract_intersection':True}) 
test74_m = np.zeros(shape = (1,12))
test74_c = np.zeros(shape = (1,12))

test74_c[0][6]-=0.02
test74_c[0][5]+=0.06

test = HOMExShapeletPair(*test74_init[:-1],**test74_init[-1])
test.setup_shapelet_psf(test74_m[0],test74_c[0],4)
truth = test.psf_light
model = test.psf_model_light

residual = model.drawImage(scale = 1.0, nx = 100, ny = 100).array - truth.drawImage(scale = 1.0, nx = 100, ny = 100).array

plt.imshow(residual)
plt.title('trefoil 2')
plt.colorbar()
plt.show()

plt.imshow(model.drawImage(scale = 1.0, nx = 100, ny = 100).array)
plt.title('trefoil 2')
#plt.colorbar()
plt.show()



In [ ]:
test81_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test81_m = np.zeros(shape = (10,12))
test81_c = np.zeros(shape = (10,12))
for i in range(test81_c.shape[0]):
    test81_c[i][4]+=0.001*i

test81result = []
for i in range(len(test81_m)):
    test = HOMExShapeletPair(*test81_init[i][:-1],**test81_init[i][-1])
    test.setup_shapelet_psf(test81_m[i],test81_c[i],4)
    results = test.get_results()
    test81result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test81_m)*100)+"%")


In [ ]:
test82_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test82_m = np.zeros(shape = (10,12))
test82_c = np.zeros(shape = (10,12))
for i in range(test82_c.shape[0]):
    test82_c[i][6]+=0.001*i

test82result = []
for i in range(len(test81_m)):
    test = HOMExShapeletPair(*test82_init[i][:-1],**test82_init[i][-1])
    test.setup_shapelet_psf(test82_m[i],test82_c[i],4)
    results = test.get_results()
    test82result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test82_m)*100)+"%")


In [ ]:
test83_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test83_m = np.zeros(shape = (10,12))
test83_c = np.zeros(shape = (10,12))
for i in range(test83_c.shape[0]):
    test83_c[i][4]+=0.001*i
    test83_c[i][6]+=0.001*i

test83result = []
for i in range(len(test83_m)):
    test = HOMExShapeletPair(*test83_init[i][:-1],**test83_init[i][-1])
    test.setup_shapelet_psf(test83_m[i],test83_c[i],4)
    results = test.get_results()
    test83result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test83_m)*100)+"%")


In [ ]:
dm1 = [t['dm'][4] for t in test81result]
dm2 = [t['dm'][6] for t in test82result]
dmtot = np.array(dm1)+np.array(dm2)

dshear1 = [t['abs_bias'][0] for t in test81result]
dshear2 = [t['abs_bias'][0] for t in test82result]
dsheartot = np.array(dshear1)+np.array(dshear2)

plt.plot(dmtot, dsheartot, label = 'dg('+r"$dm_{1,2}$"+') + dg('+r"$dm_{3,0}$"+')')
plt.plot(dmtot, [t['abs_bias'][0] for t in test83result],label = 'dg('+r"$dm_{1,2}$"+' + '+r"$dm_{3,0}$"+')')
plt.ylabel(r'$\Delta g_1$')
plt.xlabel(r'$\Delta dm_{1,2} + dm_{3,0} $')
plt.legend()

In [ ]:
test91_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test91_m = np.zeros(shape = (10,12))
test91_c = np.zeros(shape = (10,12))
for i in range(test91_c.shape[0]):
    test91_c[i][7]+=0.001*i

test91result = []
for i in range(len(test91_m)):
    test = HOMExShapeletPair(*test91_init[i][:-1],**test91_init[i][-1])
    test.setup_shapelet_psf(test91_m[i],test91_c[i],4)
    results = test.get_results()
    test91result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test91_m)*100)+"%")


In [ ]:
test92_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test92_m = np.zeros(shape = (10,12))
test92_c = np.zeros(shape = (10,12))
for i in range(test92_c.shape[0]):
    test92_c[i][8]+=0.001*i

test92result = []
for i in range(len(test92_m)):
    test = HOMExShapeletPair(*test92_init[i][:-1],**test92_init[i][-1])
    test.setup_shapelet_psf(test92_m[i],test92_c[i],4)
    results = test.get_results()
    test92result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test92_m)*100)+"%")


In [ ]:
test93_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test93_m = np.zeros(shape = (10,12))
test93_c = np.zeros(shape = (10,12))
for i in range(test93_c.shape[0]):
    test93_c[i][7]+=0.001*i
    test93_c[i][8]+=0.001*i

test93result = []
for i in range(len(test93_m)):
    test = HOMExShapeletPair(*test93_init[i][:-1],**test93_init[i][-1])
    test.setup_shapelet_psf(test93_m[i],test93_c[i],4)
    results = test.get_results()
    test93result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test93_m)*100)+"%")


In [ ]:
dm1 = [t['dm'][7] for t in test91result]
dm2 = [t['dm'][8] for t in test92result]
dmtot = np.array(dm1)+np.array(dm2)

dshear1 = [t['abs_bias'][0] for t in test91result]
dshear2 = [t['abs_bias'][0] for t in test92result]
dsheartot = np.array(dshear1)+np.array(dshear2)

plt.plot(dmtot, dsheartot, label = 'dg('+r"$dm_{4,0}$"+') + dg('+r"$dm_{3,1}$"+')')
plt.plot(dmtot, [t['abs_bias'][0] for t in test93result],label = 'dg('+r"$dm_{4,0}$"+' + '+r"$dm_{3,1}$"+')')

plt.ylabel(r'$\Delta g_1$')
plt.xlabel(r'$\Delta dm_{4,0} + dm_{3,1} $')
plt.legend()

In [ ]:
test101_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test101_m = np.zeros(shape = (10,12))
test101_c = np.zeros(shape = (10,12))
for i in range(test101_c.shape[0]):
    test101_c[i][3]+=0.003*i

test101result = []
for i in range(len(test101_m)):
    test = HOMExShapeletPair(*test101_init[i][:-1],**test101_init[i][-1])
    test.setup_shapelet_psf(test101_m[i],test101_c[i],4)
    results = test.get_results()
    test101result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test101_m)*100)+"%")


In [ ]:
test102_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test102_m = np.zeros(shape = (10,12))
test102_c = np.zeros(shape = (10,12))
for i in range(test102_c.shape[0]):
    test102_c[i][8]+=0.001*i

test102result = []
for i in range(len(test102_m)):
    test = HOMExShapeletPair(*test102_init[i][:-1],**test102_init[i][-1])
    test.setup_shapelet_psf(test102_m[i],test102_c[i],4)
    results = test.get_results()
    test102result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test102_m)*100)+"%")


In [ ]:
test103_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test103_m = np.zeros(shape = (10,12))
test103_c = np.zeros(shape = (10,12))
for i in range(test103_c.shape[0]):
    test103_c[i][8]+=0.001*i
    test101_c[i][3]+=0.003*i

test103result = []
for i in range(len(test103_m)):
    test = HOMExShapeletPair(*test103_init[i][:-1],**test103_init[i][-1])
    test.setup_shapelet_psf(test103_m[i],test103_c[i],4)
    results = test.get_results()
    test103result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test103_m)*100)+"%")


In [ ]:
dm1 = [t['dm'][3] for t in test101result]
dm2 = [t['dm'][8] for t in test102result]
dmtot = np.array(dm1)+np.array(dm2)

dshear1 = [t['abs_bias'][0] for t in test101result]
dshear2 = [t['abs_bias'][0] for t in test102result]
dsheartot = np.array(dshear1)+np.array(dshear2)

plt.plot(dmtot, dsheartot, label = 'dg('+r"$dm_{3,1}$"+') + dg('+r"$dm_{3,0}$"+')')
plt.plot(dmtot, [t['abs_bias'][0] for t in test103result],label = 'dg('+r"$dm_{3,1}$"+' + '+r"$dm_{3,0}$"+')')
plt.ylabel(r'$\Delta g_1$')
plt.xlabel(r'$\Delta dm_{1,2} + dm_{3,0} $')
plt.legend()

In [ ]:
test111_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test111_m = np.zeros(shape = (10,25))
test111_c = np.zeros(shape = (10,25))
for i in range(test111_c.shape[0]):
    test111_c[i][13]+=0.001*i

test111result = []
for i in range(len(test111_m)):
    test = HOMExShapeletPair(*test111_init[i][:-1],**test111_init[i][-1])
    test.setup_shapelet_psf(test111_m[i],test111_c[i],6)
    results = test.get_results()
    test111result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test111_m)*100)+"%")


In [ ]:
test112_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test112_m = np.zeros(shape = (10,25))
test112_c = np.zeros(shape = (10,25))
for i in range(test112_c.shape[0]):
    test112_c[i][19]+=0.001*i

test112result = []
for i in range(len(test112_m)):
    test = HOMExShapeletPair(*test112_init[i][:-1],**test112_init[i][-1])
    test.setup_shapelet_psf(test112_m[i],test112_c[i],6)
    results = test.get_results()
    test112result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test112_m)*100)+"%")


In [ ]:
test113_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test113_m = np.zeros(shape = (10,25))
test113_c = np.zeros(shape = (10,25))
for i in range(test113_c.shape[0]):
    test113_c[i][19]+=0.001*i
    test113_c[i][13]+=0.001*i


test113result = []
for i in range(len(test113_m)):
    test = HOMExShapeletPair(*test113_init[i][:-1],**test113_init[i][-1])
    test.setup_shapelet_psf(test113_m[i],test113_c[i],6)
    results = test.get_results()
    test113result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test113_m)*100)+"%")


In [ ]:
dm1 = [t['dm'][13] for t in test111result]
dm2 = [t['dm'][19] for t in test112result]
dmtot = np.array(dm1)+np.array(dm2)

dshear1 = [t['abs_bias'][0] for t in test111result]
dshear2 = [t['abs_bias'][0] for t in test112result]
dsheartot = np.array(dshear1)+np.array(dshear2)


plt.plot(dm1, dshear1)
plt.xlabel(r'$\Delta dm_{4,1} $')
plt.ylabel(r'$\Delta g_1$')

plt.show()

plt.plot(dm2, dshear2)
plt.xlabel(r'$\Delta dm_{5,1} $')
plt.ylabel(r'$\Delta g_1$')


plt.show()

plt.plot(dmtot, dsheartot, label = 'dg('+r"$dm_{4,1}$"+') + dg('+r"$dm_{5,1}$"+')')
plt.plot(dmtot, [t['abs_bias'][0] for t in test113result],label = 'dg('+r"$dm_{4,1}$"+' + '+r"$dm_{5,1}$"+')')
plt.ylabel(r'$\Delta g_1$')
plt.xlabel(r'$\Delta dm_{4,1} + dm_{5,1} $')
plt.legend()
plt.show()

In [ ]:
test121_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test121_m = np.zeros(shape = (10,25))
test121_c = np.zeros(shape = (10,25))
for i in range(test121_c.shape[0]):
    test111_c[i][3]+=0.001*i

test121result = []
for i in range(len(test121_m)):
    test = HOMExShapeletPair(*test121_init[i][:-1],**test121_init[i][-1])
    test.setup_shapelet_psf(test121_m[i],test121_c[i],6)
    results = test.get_results()
    test121result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test121_m)*100)+"%")


In [ ]:
test123_init = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(10)
]
test123_m = np.zeros(shape = (10,25))
test123_c = np.zeros(shape = (10,25))
for i in range(test113_c.shape[0]):
    test123_c[i][19]+=0.001*i
    test123_c[i][3]+=0.001*i


test123result = []
for i in range(len(test123_m)):
    test = HOMExShapeletPair(*test123_init[i][:-1],**test123_init[i][-1])
    test.setup_shapelet_psf(test123_m[i],test123_c[i],6)
    results = test.get_results()
    test123result.append(results)
    clear_output() 
    print ("Finished "+str(float((i+1))/len(test123_m)*100)+"%")


In [ ]:
dm1 = [t['dm'][3] for t in test121result]
dm2 = [t['dm'][19] for t in test112result]
dmtot = np.array(dm1)+np.array(dm2)

dshear1 = [t['abs_bias'][0] for t in test121result]
dshear2 = [t['abs_bias'][0] for t in test112result]
dsheartot = np.array(dshear1)+np.array(dshear2)


plt.plot(dm1, dshear1)
plt.xlabel(r'$\Delta dm_{3,0} $')
plt.ylabel(r'$\Delta g_1$')

plt.show()

plt.plot(dm2, dshear2)
plt.xlabel(r'$\Delta dm_{5,1} $')
plt.ylabel(r'$\Delta g_1$')


plt.show()

plt.plot(dmtot, dsheartot, label = 'dg('+r"$dm_{4,1}$"+') + dg('+r"$dm_{5,1}$"+')')
plt.plot(dmtot, [t['abs_bias'][0] for t in test113result],label = 'dg('+r"$dm_{4,1}$"+' + '+r"$dm_{5,1}$"+')')
plt.ylabel(r'$\Delta g_1$')
plt.xlabel(r'$\Delta dm_{4,1} + dm_{5,1} $')
plt.legend()
plt.show()

In [ ]:
test13_init = [("gaussian" ,0.5+0.1*i    ,0.1,e2(0.1,0.28),1e-8,1e-8,"gaussian"  , 1.5  ,{'subtract_intersection':True}) for i in range(40)
]
test13_m = np.zeros(shape = (22,40,25))
test13_c = np.zeros(shape = (22,40,25))
for index in range(22):
    for i in range(40):
        test13_c[index][i][index+3]+=0.005



In [ ]:
test13result = []
for i in range(len(test13_m)):
    print( "Start tests for moment"+ str(i+4))
    test13result.append(do_tests_speed(test13_init,i,test13_m[i],test13_c[i],6))


In [ ]:
from scipy import interpolate

In [ ]:
import pickle


with open("../plots2/pickle/add_size_ratio.pkl","wb") as f:
    pickle.dump([pqlist,test13result,test13_init, test131result, test131_init],f)



In [ ]:
spine_list1 = []
spine_list2 = []
pq4nersc = []


pqlist = test1.sxm.get_pq_full(6)[3:]
fig = plt.figure(figsize = (21,12))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)
for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    ax = plt.subplot(4,7,1+7*(n-3)+p)
    
    size_ratio = np.array([t['gal_sigma']/t['psf_sigma'] for t in test13result[j]])
    dg1 = np.array([t["abs_bias"][0] for t in test13result[j]])/0.005
    dg2 = np.array([t["abs_bias"][1] for t in test13result[j]])/0.005
    
    plt.plot(size_ratio,dg1,label='g1')
    plt.plot(size_ratio,dg2,label='g2')
    
    #print test4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    plt.ticklabel_format(axis='y',style='sci',scilimits=(0,3))
    
    spine_list1.append(dg1)
    spine_list2.append(dg2)
    pq4nersc.append([p,q])
    
    
    
 
    plt.xlabel(r"$\sigma_{galaxy}/\sigma_{PSF}$")
    plt.ylabel(r'$\delta g_i / \delta m_{p,q}$')
    
    plt.title(str((p,q)))
    #plt.show()
    plt.legend()
    
#fig.colorbar(axes)

In [ ]:
np.save('Results/size_ratio.npy',size_ratio)
np.save('Results/dg1.npy',np.array(spine_list1))
np.save('Results/dg2.npy',np.array(spine_list2))
np.save('Results/pq4nersc.npy', np.array(pq4nersc))

In [ ]:
test131_init = [("sersic" ,0.5+0.1*i    ,0.1,e2(0.1,0.28),1e-8,1e-8,"gaussian"  ,1.5  ,{'subtract_intersection':True,'sersicn':3.0}) for i in range(40)
]
test131_m = np.zeros(shape = (22,40,25))
test131_c = np.zeros(shape = (22,40,25))
for index in range(22):
    for i in range(40):
        test131_c[index][i][index+3]+=0.005



In [ ]:
test131result = []
for i in range(len(test131_m)):
    print( "Start tests for moment"+ str(i+4))
    test131result.append(do_tests_speed(test131_init,i,test131_m[i],test131_c[i],6))


In [ ]:
test132_init = [("sersic" ,1.0+0.2*i    ,0.1,e2(0.1,0.28),1e-8,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':0.5}) for i in range(40)
]
test132_m = np.zeros(shape = (22,40,25))
test132_c = np.zeros(shape = (22,40,25))
for index in range(22):
    for i in range(40):
        test132_c[index][i][index+3]+=0.005



In [ ]:
test132result = []
for i in range(len(test132_m)):
    print "Start tests for moment"+ str(i+4)
    test132result.append(do_tests_speed(test132_init,i,test132_m[i],test132_c[i],6))


In [ ]:
%store test13result
%store test131result
%store test132result

In [ ]:
%store -r test13result
%store -r test131result
%store -r test132result

In [ ]:
y_range = {}
for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    
    if n not in y_range.keys():
        y_range[n] = [0,0]
    #print min(min(np.array([t["abs_bias"][0] for t in test13result[j]])/0.005),y_range[n][0])
    y_range[n][0] = min(min(np.array([t["abs_bias"][0] for t in test13result[j]])/0.005)*1.1,y_range[n][0])
    y_range[n][0] = min(min(np.array([t["abs_bias"][1] for t in test13result[j]])/0.005)*1.1,y_range[n][0])
    y_range[n][0] = min(min(np.array([t["abs_bias"][0] for t in test131result[j]])/0.005)*1.1,y_range[n][0])
    y_range[n][0] = min(min(np.array([t["abs_bias"][1] for t in test131result[j]])/0.005)*1.1,y_range[n][0])
    
    y_range[n][1] = max(max(np.array([t["abs_bias"][0] for t in test13result[j]])/0.005)*1.1,y_range[n][1])
    y_range[n][1] = max(max(np.array([t["abs_bias"][1] for t in test13result[j]])/0.005)*1.1,y_range[n][1])
    y_range[n][1] = max(max(np.array([t["abs_bias"][0] for t in test131result[j]])/0.005)*1.1,y_range[n][1])
    y_range[n][1] = max(max(np.array([t["abs_bias"][1] for t in test131result[j]])/0.005)*1.1,y_range[n][1])
    

In [ ]:
print y_range

In [ ]:


pqlist = test1.sxm.get_pq_full(6)[3:]
fig = plt.figure(figsize = (21,12))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.0, hspace=0.0)

# f, axes = plt.subplots(4, 7, sharex='col', sharey='row', figsize=(21,12))
# f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.0, hspace=0.0)




for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    #print n
    ax = plt.subplot(4,7,1+7*(n-3)+p)
    
    
    
    plt.plot(np.array([t['gal_hlr']/t['psf_hlr'] for t in test13result[j]]),np.array([t["abs_bias"][0] for t in test13result[j]])/0.005,color = 'blue')
    plt.plot(np.array([t['gal_hlr']/t['psf_hlr'] for t in test13result[j]]),np.array([t["abs_bias"][1] for t in test13result[j]])/0.005,color = 'orange')

    plt.plot(np.array([t['gal_hlr']/t['psf_hlr'] for t in test131result[j]]),np.array([t["abs_bias"][0] for t in test131result[j]])/0.005,'--',color = 'blue')
    plt.plot(np.array([t['gal_hlr']/t['psf_hlr'] for t in test131result[j]]),np.array([t["abs_bias"][1] for t in test131result[j]])/0.005,'--',color = 'orange')
    
#     plt.plot(np.array([t['gal_hlr']/t['psf_hlr'] for t in test132result[j]]),np.array([t["abs_bias"][0] for t in test131result[j]])/0.005,'.-',color = 'blue')
#     plt.plot(np.array([t['gal_hlr']/t['psf_hlr'] for t in test132result[j]]),np.array([t["abs_bias"][1] for t in test131result[j]])/0.005,'.-',color = 'orange')
    
    
    #print test4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    
    
    
    ax.tick_params(
        axis='x',          # changes apply to the x-axis
        direction = 'in',
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=True,         # ticks along the top edge are off
        labelbottom=False)
    
    ax.tick_params(
        axis='y',          # changes apply to the x-axis
        direction = 'in',
        which='both',      # both major and minor ticks are affected
        left=True,      # ticks along the bottom edge are off
        right=False,         # ticks along the top edge are off
        labelleft=False)
    
    #ax.tick_params(axis="y",direction="in")
    
    
    
    if j in list(range(15,22)):
        plt.xlabel(r"$R_h^{galaxy}/R_h^{PSF}$")
        ax.tick_params(
            axis='x',          # changes apply to the x-axis
            direction = 'in',
            which='both',      # both major and minor ticks are affected
            bottom=True,      # ticks along the bottom edge are off
            top=True,         # ticks along the top edge are off
            labelbottom=True)
    if j in [0,4,9,15]:
        plt.ylabel(r'$\delta g_i / \delta m_{p,q}$')
        plt.ticklabel_format(axis='y',style='scientific',scilimits=(0,3))
        ax.tick_params(
            axis='y',          # changes apply to the x-axis
            direction = 'in',
            which='both',      # both major and minor ticks are affected
            left=True,      # ticks along the bottom edge are off
            right=False,         # ticks along the top edge are off
            labelleft=True)
    
    plt.ylim(y_range[n])
    
    plt.title(str((p,q)),y = 0.8)
    #plt.show()
    #plt.legend([])
    

plt.subplot(4,7,7,frame_on = False)
plt.plot([0],[0],color = 'blue',label = r'Gaussian $g_1$')
plt.plot([0],[0],color = 'orange',label = r'Gaussian $g_2$')
plt.plot([0],[0],'--',color = 'blue',label = r'Sersic n = 3.0 $g_1$')
plt.plot([0],[0],'--',color = 'orange',label = r'Sersic n = 3.0 $g_2$')
plt.axis('off')
plt.legend(fontsize = 'large',frameon = False)

    
#fig.colorbar(axes)

In [ ]:
print np.array(pq4nersc)

In [ ]:
def linearity_check(m1, dm1, m2, dm2, config, n_max = 6):
    
    vector_length = (n_max +1 + 3) * (n_max - 1) / 2
    
    test1_m = np.zeros(shape = (1,vector_length))
    test1_c = np.zeros(shape = (1,vector_length))
    test1_c[0][m1]+=dm1
        

    test1 = HOMExShapeletPair(*config[0][:-1],**config[0][-1])
    test1.setup_shapelet_psf(test1_m[0],test1_c[0],n_max)
    results1 = test1.get_results()
            
    test2_m = np.zeros(shape = (1,vector_length))
    test2_c = np.zeros(shape = (1,vector_length))
    test2_c[0][m2]+=dm2
   

    test2 = HOMExShapeletPair(*config[0][:-1],**config[0][-1])
    test2.setup_shapelet_psf(test2_m[0],test2_c[0],n_max)
    results2 = test2.get_results()
    
    
    test3_m = np.zeros(shape = (1,vector_length))
    test3_c = np.zeros(shape = (1,vector_length))
    test3_c[0][m1]+=dm1
    test3_c[0][m2]+=dm2
   

    test3 = HOMExShapeletPair(*config[0][:-1],**config[0][-1])
    test3.setup_shapelet_psf(test3_m[0],test3_c[0],n_max)
    results3 = test3.get_results()
    
    
    dshear1 = results1['abs_bias'][0]
    dshear2 = results2['abs_bias'][0]
    
    
    #print dshear1, dshear2
        
    linear_results = dshear1 + dshear2
    
    auto_results = results3['abs_bias'][0]
    #print results3['actual_dm']
    #print linear_results, auto_results
    
    error_over_minor = abs(linear_results -  auto_results) / min(np.abs(dshear1) , np.abs(dshear2) )
    error_over_sum = abs(linear_results -  auto_results) / (np.abs(dshear1) + np.abs(dshear2))
    
    
    return error_over_minor, error_over_sum
    
    
    
    
    
    

    

In [ ]:
config = [("gaussian" ,3.0    ,0.28,0.28,0.001,0.001,"gaussian"  ,2.0 ,{'subtract_intersection':True}) for i in range(1)]


error_over_minor_matrix = np.zeros(shape = (12,12))
error_over_sum_matrix = np.zeros(shape = (12,12))


for i in range(12):
    for j in range(i,12):
        print i,j
        eom, eos = linearity_check(i,0.001,j,0.001,config,4)
        
        error_over_minor_matrix[i][j] = eom
        error_over_sum_matrix[i][j] = eos




In [ ]:
n_max = 4



dg_scale = []
for i in range(12):
    print i
    vector_length = (n_max +1 + 3) * (n_max - 1) / 2
    
    test1_m = np.zeros(shape = (1,vector_length))
    test1_c = np.zeros(shape = (1,vector_length))
    test1_c[0][i]+=0.001
    
    
    test1 = HOMExShapeletPair(*config[0][:-1],**config[0][-1])
    test1.setup_shapelet_psf(test1_m[0],test1_c[0],n_max)
    results1 = test1.get_results()
    dg_scale.append(np.abs(results1['abs_bias'][0]))

In [ ]:
pqlist = test1.sxm.get_pq_full(4)


label_list = []
for i in range(12):

    label_list.append("m"+str(pqlist[i][0])+str(pqlist[i][1])) 
    

fig, ax = plt.subplots(1,1,figsize=(8, 8)) 
mappable = ax.imshow(error_over_minor_matrix, cmap = 'Blues',vmin = -0.0, vmax = 0.5)

# Set number of ticks for x-axis
# Set ticks labels for x-axis
ax.set_xticks(np.arange(0,12,1))
ax.set_yticks(np.arange(0,12,1))

ax.set_xticklabels(label_list, rotation='vertical', fontsize=14)
ax.set_yticklabels(label_list, rotation='horizontal', fontsize=14)

plt.colorbar(mappable, ax = ax, label = r"$   \frac{dg(dm_1) + dg_2(dm_2) - dg(dm_1+dm_2)}{min(dg(dm_1), dg(dm_2))}$")

plt.title(r"$   \frac{dg(dm_1) + dg_2(dm_2) - dg(dm_1+dm_2)}{min(dg(dm_1), dg(dm_2))}$")

In [ ]:
pqlist = test1.sxm.get_pq_full(4)


label_list = []
for i in range(12):

    label_list.append("m"+str(pqlist[i][0])+str(pqlist[i][1])) 
    

fig, ax = plt.subplots(1,1,figsize=(8, 8)) 
mappable = ax.imshow(error_over_sum_matrix, cmap = 'Blues',vmin = -0.0, vmax = 1.0)

# Set number of ticks for x-axis
# Set ticks labels for x-axis
ax.set_xticks(np.arange(0,12,1))
ax.set_yticks(np.arange(0,12,1))

ax.set_xticklabels(label_list, rotation='vertical', fontsize=14)
ax.set_yticklabels(label_list, rotation='horizontal', fontsize=14)

plt.colorbar(mappable, ax = ax, label = r"$   \frac{dg(dm_1) + dg_2(dm_2) - dg(dm_1+dm_2)}{dg(dm_1) +  dg(dm_2)}$")

plt.title(r"$   \frac{dg(dm_1) + dg_2(dm_2) - dg(dm_1+dm_2)}{dg(dm_1) +  dg(dm_2)}$")
plt.show()

fig, ax = plt.subplots(1,1,figsize=(6, 4)) 
mappable = plt.plot(np.arange(0,12), dg_scale,'+')
plt.yscale('log')

ax.axvspan(3, 6, color='r', alpha=0.2, lw=0)

ax.set_xticks(np.arange(0,12,1))
ax.set_xticklabels(label_list, rotation='vertical', fontsize=14)
plt.ylabel('dg1')

plt.show()






In [ ]:
print pqlist
pq_for_m = [4,6,8,15,17,19,21]

In [ ]:
test14_init = [("gaussian" ,1.0+0.2*i    ,0.1,0.26,1e-8,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True}) for i in range(40)
]
test14_m = np.zeros(shape = (7,40,25))
test14_c = np.zeros(shape = (7,40,25))
for index in range(7):
    for i in range(40):
        test14_c[index][i][pq_for_m[index]+3]+=0.005

        


In [ ]:
test14result = []
for i in range(len(test14_m)):
    print "Start tests for moment"+ str(pq_for_m[i]+4)
    test14result.append(do_tests(test14_init,i,test14_m[i],test14_c[i],6))


In [ ]:
test141_init = [("gaussian" ,1.0+0.2*i    ,0.1,0.26,0.01,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True}) for i in range(40)
]
test141_m = np.zeros(shape = (7,40,25))
test141_c = np.zeros(shape = (7,40,25))
for index in range(7):
    for i in range(40):
        test141_c[index][i][pq_for_m[index]+3]+=0.005

        


In [ ]:
test141result = []
for i in range(len(test141_m)):
    print "Start tests for moment"+ str(pq_for_m[i]+4)
    test141result.append(do_tests(test141_init,i,test141_m[i],test141_c[i],6))


In [ ]:
test142_init = [("gaussian" ,1.0+0.2*i    ,0.1,0.26,1e-8,0.01,"gaussian"  ,3.0  ,{'subtract_intersection':True}) for i in range(40)
]
test142_m = np.zeros(shape = (7,40,25))
test142_c = np.zeros(shape = (7,40,25))
for index in range(7):
    for i in range(40):
        test142_c[index][i][pq_for_m[index]+3]+=0.005

        


In [ ]:
test142result = []
for i in range(len(test142_m)):
    print "Start tests for moment"+ str(pq_for_m[i]+4)
    test142result.append(do_tests(test142_init,i,test142_m[i],test142_c[i],6))


In [ ]:
print test14result[0][0]

In [ ]:
size_ratio = np.zeros(shape = (40))
m1_size = np.zeros(shape = (7,40))
m2_size = np.zeros(shape = (7,40))

for i in range(40):
    size_ratio[i] = test14result[0][i]['gal_sigma']/test14result[0][i]['psf_sigma']
    for j in range(7):
        m1_size[j][i] = (test141result[j][i]['abs_bias'][0] - test14result[j][i]['abs_bias'][0])/0.01/0.005
        m2_size[j][i] = (test142result[j][i]['abs_bias'][1] - test14result[j][i]['abs_bias'][1])/0.01/0.005

In [ ]:
print m1_size.shape

In [ ]:
np.save('data/multiplicative_size_ratio',size_ratio)
np.save('data/m1_size_ratio',m1_size)
np.save('data/m2_size_ratio',m2_size)


In [ ]:

fig = plt.figure(figsize = (21,4))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)
for j in range(7):
    p,q = pqlist[pq_for_m[j]][0],pqlist[pq_for_m[j]][1]
    n = p+q
    ax = plt.subplot(1,7,j+1)
    
    m1 = m1_size[j]
    m2 = m2_size[j]
    
    
    plt.plot(size_ratio,m1,label='g1')
    plt.plot(size_ratio,m2,label='g2')
    
    #print test4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    plt.ticklabel_format(axis='y',style='sci',scilimits=(0,3))
    

    
 
    plt.xlabel(r"$\sigma_{galaxy}/\sigma_{PSF}$")
    plt.ylabel(r'$ m / B[ \mathbf{m}_{p,q}]$')
    
    plt.title(str((p,q)))
    #plt.show()
    plt.legend()
    
#fig.colorbar(axes)

In [ ]:
test15_init = [("sersic" ,1.0+0.2*i    ,0.1,0.26,1e-8,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':0.5}) for i in range(40)
]+[("sersic" ,1.0+0.2*i    ,0.1,0.26,0.01,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':0.5}) for i in range(40)
]+[("sersic" ,1.0+0.2*i    ,0.1,0.26,1e-8,0.01,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':0.5}) for i in range(40)
]
test15_m = np.zeros(shape = (7,120,25))
test15_c = np.zeros(shape = (7,120,25))
for index in range(7):
    for i in range(120):
        test15_c[index][i][pq_for_m[index]+3]+=0.005

        


In [ ]:
test15result = []
for i in range(len(test15_m)):
    print "Start tests for moment"+ str(pq_for_m[i]+4)
    test15result.append(do_tests(test15_init,i,test15_m[i],test15_c[i],6))


In [ ]:
test151_init = [("sersic" ,1.0+0.2*i    ,0.1,0.26,1e-8,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':3.0}) for i in range(40)
]+[("sersic" ,1.0+0.2*i    ,0.1,0.26,0.01,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':3.0}) for i in range(40)
]+[("sersic" ,1.0+0.2*i    ,0.1,0.26,1e-8,0.01,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':3.0}) for i in range(40)
]
test151_m = np.zeros(shape = (7,120,25))
test151_c = np.zeros(shape = (7,120,25))
for index in range(7):
    for i in range(120):
        test151_c[index][i][pq_for_m[index]+3]+=0.005

        


In [ ]:
test151result = []
for i in range(len(test151_m)):
    print "Start tests for moment"+ str(pq_for_m[i]+4)
    test151result.append(do_tests(test151_init,i,test151_m[i],test151_c[i],6))


In [ ]:
size_ratio_gau = np.zeros(shape = (40))
m1_size_gau = np.zeros(shape = (7,40))
m2_size_gau = np.zeros(shape = (7,40))

for i in range(40):
    size_ratio_gau[i] = test15result[0][i]['gal_hlr']/test15result[0][i]['psf_hlr']
    for j in range(7):
        m1_size_gau[j][i] = (test15result[j][i+40]['abs_bias'][0] - test15result[j][i]['abs_bias'][0])/0.01/0.005
        m2_size_gau[j][i] = (test15result[j][i+80]['abs_bias'][1] - test15result[j][i]['abs_bias'][1])/0.01/0.005

In [ ]:
size_ratio_ser = np.zeros(shape = (40))
m1_size_ser = np.zeros(shape = (7,40))
m2_size_ser = np.zeros(shape = (7,40))

for i in range(40):
    size_ratio_ser[i] = test151result[0][i]['gal_hlr']/test151result[0][i]['psf_hlr']
    for j in range(7):
        m1_size_ser[j][i] = (test151result[j][i+40]['abs_bias'][0] - test151result[j][i]['abs_bias'][0])/0.01/0.005
        m2_size_ser[j][i] = (test151result[j][i+80]['abs_bias'][1] - test151result[j][i]['abs_bias'][1])/0.01/0.005

In [ ]:
y_range_15 = {}
for j in range(7):
    p,q = pqlist[pq_for_m[j]][0],pqlist[pq_for_m[j]][1]
    n = p+q
    
    if n not in y_range_15.keys():
        y_range_15[n] = [0,0]
    #print min(min(np.array([t["abs_bias"][0] for t in test13result[j]])/0.005),y_range[n][0])
    y_range_15[n][0] = min(min(m1_size_gau[j]*1.1),y_range_15[n][0])
    y_range_15[n][0] = min(min(m2_size_gau[j]*1.1),y_range_15[n][0])
    y_range_15[n][0] = min(min(m1_size_ser[j]*1.1),y_range_15[n][0])
    y_range_15[n][0] = min(min(m2_size_ser[j]*1.1),y_range_15[n][0])
    
    y_range_15[n][1] = max(max(m1_size_gau[j]*1.1),y_range_15[n][1])
    y_range_15[n][1] = max(max(m2_size_gau[j]*1.1),y_range_15[n][1])
    y_range_15[n][1] = max(max(m1_size_ser[j]*1.1),y_range_15[n][1])
    y_range_15[n][1] = max(max(m2_size_ser[j]*1.1),y_range_15[n][1])
    

In [ ]:
print y_range_15

In [ ]:

fig = plt.figure(figsize = (12,6))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0, hspace=0)

for j in range(7):
    p,q = pqlist[pq_for_m[j]][0],pqlist[pq_for_m[j]][1]
    n = p+q
    
    position = 1+j
    if j>2: position = 2+j
    ax = plt.subplot(2,4,position)
    
    
    
    plt.plot(size_ratio_gau,m1_size_gau[j],'--',color = 'blue',label = r'Gaussian $g_1$')
    plt.plot(size_ratio_gau,m2_size_gau[j],'--',color = 'orange',label = r'Gaussian $g_2$')
    
    plt.plot(size_ratio_ser,m1_size_ser[j],color = 'blue',label = r'Sersic n=3 $g_1$')
    plt.plot(size_ratio_ser,m2_size_ser[j],color = 'orange',label = r'Sersic n=3 $g_2$')
    
    #print test4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    plt.ticklabel_format(axis='y',style='sci',scilimits=(0,3))
    
        
    
    ax.tick_params(
        axis='x',          # changes apply to the x-axis
        direction = 'in',
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=True,         # ticks along the top edge are off
        labelbottom=False)
    
    ax.tick_params(
        axis='y',          # changes apply to the x-axis
        direction = 'in',
        which='both',      # both major and minor ticks are affected
        left=True,      # ticks along the bottom edge are off
        right=False,         # ticks along the top edge are off
        labelleft=False)
    
    #ax.tick_params(axis="y",direction="in")
    
    
    
    if j in list(range(3,7)):
        plt.xlabel(r"$\sigma_{galaxy}/\sigma_{PSF}$")
        ax.tick_params(
            axis='x',          # changes apply to the x-axis
            direction = 'in',
            which='both',      # both major and minor ticks are affected
            bottom=True,      # ticks along the bottom edge are off
            top=True,         # ticks along the top edge are off
            labelbottom=True)
    if j in [0,3]:
        plt.ylabel(r'$ m / B[ \mathbf{m}_{p,q}]$')
        plt.ticklabel_format(axis='y',style='scientific',scilimits=(0,3))
        ax.tick_params(
            axis='y',          # changes apply to the x-axis
            direction = 'in',
            which='both',      # both major and minor ticks are affected
            left=True,      # ticks along the bottom edge are off
            right=False,         # ticks along the top edge are off
            labelleft=True)
    
    plt.ylim(y_range_15[n])

    #plt.xlabel(r"$\sigma_{galaxy}/\sigma_{PSF}$")
    #plt.ylabel(r'$ m / B[ \mathbf{m}_{p,q}]$')
    
    plt.title(str((p,q)),y = 0.8)
    #plt.show()
    #plt.legend()
    
plt.subplot(2,4,4,frame_on = False)
plt.plot([0],[0],'--',color = 'blue',label = r'Gaussian $g_1$')
plt.plot([0],[0],'--',color = 'orange',label = r'Gaussian $g_2$')
plt.plot([0],[0],color = 'blue',label = r'Sersic n = 3.0 $g_1$')
plt.plot([0],[0],color = 'orange',label = r'Sersic n = 3.0 $g_2$')
plt.axis('off')
plt.legend(fontsize = 'medium',frameon = False)


In [ ]:
psf = galsim.Gaussian(sigma = 1.0)
image = psf.drawImage(scale = 0.1,method = 'no_pixel')
FWHM = psf.calculateFWHM()

In [ ]:
test17_init = [("gaussian" ,0.5+0.1*i    ,0.1,0.26,1e-8,1e-8,"gaussian"  ,1.5 ,{'subtract_intersection':True}) for i in range(40)
]+[("gaussian" ,0.5+0.1*i    ,0.1,0.26,0.01,1e-8,"gaussian"  ,1.5 ,{'subtract_intersection':True}) for i in range(40)
]+[("gaussian" ,0.5+0.1*i    ,0.1,0.26,1e-8,0.01,"gaussian"  ,1.5 ,{'subtract_intersection':True}) for i in range(40)
]
test17_m = np.zeros(shape = (22,120,25))
test17_c = np.zeros(shape = (22,120,25))
for index in range(22):
    for i in range(120):
        test17_c[index][i][index+3]+=0.005

        


In [ ]:
test17result = []
for i in range(len(test17_m)):
    print( "Start tests for moment"+ str(i+4))
    test17result.append(do_tests_speed(test17_init,i,test17_m[i],test17_c[i],6))


In [ ]:
size_ratio_gau = np.zeros(shape = (40))
m1_size_gau = np.zeros(shape = (22,40))
m2_size_gau = np.zeros(shape = (22,40))

for i in range(40):
    size_ratio_gau[i] = test17result[0][i]['gal_hlr']/test17result[0][i]['psf_hlr']
    for j in range(22):
        m1_size_gau[j][i] = (test17result[j][i+40]['abs_bias'][0] - test17result[j][i]['abs_bias'][0])/0.01/0.005
        m2_size_gau[j][i] = (test17result[j][i+80]['abs_bias'][1] - test17result[j][i]['abs_bias'][1])/0.01/0.005

In [ ]:
size_ratio_ser = np.zeros(shape = (40))
m1_size_ser = np.zeros(shape = (22,40))
m2_size_ser = np.zeros(shape = (22,40))

for i in range(40):
    size_ratio_ser[i] = test171result[0][i]['gal_hlr']/test171result[0][i]['psf_hlr']
    for j in range(22):
        m1_size_ser[j][i] = (test171result[j][i+40]['abs_bias'][0] - test171result[j][i]['abs_bias'][0])/0.01/0.005
        m2_size_ser[j][i] = (test171result[j][i+80]['abs_bias'][1] - test171result[j][i]['abs_bias'][1])/0.01/0.005

In [ ]:
y_range_15 = {}
for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    
    if n not in y_range_15.keys():
        y_range_15[n] = [0,0]
    #print min(min(np.array([t["abs_bias"][0] for t in test13result[j]])/0.005),y_range[n][0])
    y_range_15[n][0] = min(min(m1_size_gau[j]*1.1),y_range_15[n][0])
    y_range_15[n][0] = min(min(m2_size_gau[j]*1.1),y_range_15[n][0])
    y_range_15[n][0] = min(min(m1_size_ser[j]*1.1),y_range_15[n][0])
    y_range_15[n][0] = min(min(m2_size_ser[j]*1.1),y_range_15[n][0])
    
    y_range_15[n][1] = max(max(m1_size_gau[j]*1.1),y_range_15[n][1])
    y_range_15[n][1] = max(max(m2_size_gau[j]*1.1),y_range_15[n][1])
    y_range_15[n][1] = max(max(m1_size_ser[j]*1.1),y_range_15[n][1])
    y_range_15[n][1] = max(max(m2_size_ser[j]*1.1),y_range_15[n][1])
    

In [ ]:
with open("../plots2/pickle/mul_size_ratio.pkl","wb") as f:
    pickle.dump([pqlist,test17result,test171result ],f)

In [ ]:
with open('../plots2/pickle/mul_size_ratio.pkl','rb') as f:
    pqlist,test17result,test171result = pickle.load(f)

In [ ]:


pqlist = test1.sxm.get_pq_full(6)[3:]
fig = plt.figure(figsize = (21,12))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.0, hspace=0.0)

# f, axes = plt.subplots(4, 7, sharex='col', sharey='row', figsize=(21,12))
# f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.0, hspace=0.0)




for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    #print n
    ax = plt.subplot(4,7,1+7*(n-3)+p)
    
    
    
    
    plt.plot(size_ratio_gau,m1_size_gau[j],color = 'blue',label = r'Gaussian $g_1$')
    plt.plot(size_ratio_gau,m2_size_gau[j],color = 'orange',label = r'Gaussian $g_2$')
    
    plt.plot(size_ratio_ser,m1_size_ser[j],'--',color = 'blue',label = r'Gaussian $g_1$')
    plt.plot(size_ratio_ser,m2_size_ser[j],'--',color = 'orange',label = r'Gaussian $g_2$')
    
#     plt.plot(np.array([t['gal_hlr']/t['psf_hlr'] for t in test132result[j]]),np.array([t["abs_bias"][0] for t in test131result[j]])/0.005,'.-',color = 'blue')
#     plt.plot(np.array([t['gal_hlr']/t['psf_hlr'] for t in test132result[j]]),np.array([t["abs_bias"][1] for t in test131result[j]])/0.005,'.-',color = 'orange')
    
    
    #print test4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    
    
    
    ax.tick_params(
        axis='x',          # changes apply to the x-axis
        direction = 'in',
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=True,         # ticks along the top edge are off
        labelbottom=False)
    
    ax.tick_params(
        axis='y',          # changes apply to the x-axis
        direction = 'in',
        which='both',      # both major and minor ticks are affected
        left=True,      # ticks along the bottom edge are off
        right=False,         # ticks along the top edge are off
        labelleft=False)
    
    #ax.tick_params(axis="y",direction="in")
    
    
    
    if j in list(range(15,22)):
        plt.xlabel(r"$R_h^{galaxy}/R_h^{PSF}$")
        ax.tick_params(
            axis='x',          # changes apply to the x-axis
            direction = 'in',
            which='both',      # both major and minor ticks are affected
            bottom=True,      # ticks along the bottom edge are off
            top=True,         # ticks along the top edge are off
            labelbottom=True)
    if j in [0,4,9,15]:
        plt.ylabel(r'$\delta g_i / \delta m_{p,q}$')
        plt.ticklabel_format(axis='y',style='scientific',scilimits=(0,3))
        ax.tick_params(
            axis='y',          # changes apply to the x-axis
            direction = 'in',
            which='both',      # both major and minor ticks are affected
            left=True,      # ticks along the bottom edge are off
            right=False,         # ticks along the top edge are off
            labelleft=True)
    
    plt.ylim(y_range_15[n])
    
    plt.title(str((p,q)),y = 0.8)
    #plt.show()
    #plt.legend([])
    

plt.subplot(4,7,7,frame_on = False)
plt.plot([0],[0],color = 'blue',label = r'Gaussian $g_1$')
plt.plot([0],[0],color = 'orange',label = r'Gaussian $g_2$')
plt.plot([0],[0],'--',color = 'blue',label = r'Sersic n = 3.0 $g_1$')
plt.plot([0],[0],'--',color = 'orange',label = r'Sersic n = 3.0 $g_2$')
plt.axis('off')
plt.legend(fontsize = 'large',frameon = False)

    
#fig.colorbar(axes)

In [ ]:
test171_init = [("sersic" ,1.0+0.2*i    ,0.1,0.26,1e-8,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':3.0}) for i in range(40)
]+[("sersic" ,1.0+0.2*i    ,0.1,0.26,0.01,1e-8,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':3.0}) for i in range(40)
]+[("sersic" ,1.0+0.2*i    ,0.1,0.26,1e-8,0.01,"gaussian"  ,3.0  ,{'subtract_intersection':True,'sersicn':3.0}) for i in range(40)
]
test171_m = np.zeros(shape = (22,120,25))
test171_c = np.zeros(shape = (22,120,25))
for index in range(22):
    for i in range(120):
        test171_c[index][i][index+3]+=0.005

        


In [ ]:
test171result = []
for i in range(len(test171_m)):
    print( "Start tests for moment"+ str(i+4))
    test171result.append(do_tests_speed(test171_init,i,test171_m[i],test171_c[i],6))


In [ ]:
size_ratio_cosmos = np.load('data/size_ratio_array.npy')
size_ratio_cosmos = size_ratio_cosmos[size_ratio_cosmos<2.9]

In [ ]:
print(size_ratio_gau)

In [ ]:
HSC_moment_bias = np.load('data/mean_residual.npy')


In [ ]:
from scipy import interpolate

g1_m = []; g2_m = []

for i in range(22):
#     this_f1 = interpolate.LinearNDInterpolator(x, dg1[i])
#     this_f2 = interpolate.LinearNDInterpolator(x, dg2[i])

    
    this_f1 = interpolate.interp1d(size_ratio_gau, m1_size_gau[i])
    m1 = this_f1(size_ratio_cosmos)
    g1_m.append(np.mean(m1) * HSC_moment_bias[i+3])
    
    this_f2 = interpolate.interp1d(size_ratio_gau, m2_size_gau[i])
    m2 = this_f2(size_ratio_cosmos)
    g2_m.append(np.mean(m2)  * HSC_moment_bias[i+3]   )

In [ ]:
nob = 50

label_list = []
pqlist = test1.sxm.get_pq_full(6)
for i in range(nob):
    if i < 25:
        i_pre = 't'
        
    else:
        i_pre = 'r'
        
    
    label1 = i_pre+str(pqlist[i%25][0])+str(pqlist[i%25][1])
    label_list.append(label1)

fig, ax = plt.subplots(1,1,figsize=(8, 6)) 
ax.plot(np.arange(3,25),g1_m,'o',label = 'm1')
ax.plot(np.arange(3,25),g2_m,'o',label = 'm2')

ax.axvspan(6.5, 11.5, color='r', alpha=0.2, lw=0)

ax.axvspan(17.5, 24.5, color='r', alpha=0.2, lw=0)


ax.set_xticks(np.arange(3,25,1))
ax.set_xticklabels(label_list[28:], rotation='vertical', fontsize=14)

plt.grid()
plt.legend()
plt.ylabel("Multiplicative Bias")




print( "m1 = " + str(np.sum(g1_m)))
print( "m2 = " + str(np.sum(g2_m)))

In [ ]:
import pickle

    with open("../plots2/pickle/mul_prelim.pkl","wb") as f:
        pickle.dump([g1_m,g2_m,label_list ],f)

In [ ]:
psf = galsim.Gaussian(sigma = 1.5)
image = psf.drawImage(scale = 1.0, method = 'no_pixel')
print(image.calculateFWHM()*0.2)

In [ ]:
pixel_size = [0.1,0.15, 0.2,0.25, 0.3]
#gal_size = 0.17 arcsec, psf_size = 0.24 arcsec, pixel_size = 0.2 arcsec
test18_init = [("gaussian" ,0.5/this_pixel    ,0.28,0.28,1e-8,1e-8,"gaussian"  ,0.3/this_pixel ,{'subtract_intersection':True}) for this_pixel in pixel_size
]

test18_m = np.zeros(shape = (22,5,25))
test18_c = np.zeros(shape = (22,5,25))
for index in range(22):
    for i in range(5):
        #test3_c[index][i][index+3]+=HSC_moment_bias[index+3]
        test18_c[index][i][index+3]+=0.01



In [ ]:
test18result = []
for i in range(len(test18_m)):
    print( "Start tests for moment"+ str(i+4))
    test18result.append(do_tests(test18_init,i,test18_m[i],test18_c[i],6))
 

In [ ]:
pqlist = test1.sxm.get_pq_full(6)[3:]
fig = plt.figure(figsize = (21,12))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)



for j in range(22):
    p,q = pqlist[j][0],pqlist[j][1]
    n = p+q
    ax = plt.subplot(4,7,1+7*(n-3)+p)
    

    
    dg1 = np.array([t["abs_bias"][0] for t in test18result[j]])
    dg2 = np.array([t["abs_bias"][1] for t in test18result[j]])
    

    
    plt.plot(pixel_size,dg1,'.',label='g1')
    plt.plot(pixel_size,dg2,'.',label='g2')

    
    
    plt.ticklabel_format(axis='y',style='sci',scilimits=(0,3))

    #print test4_gaussian_results[j][0]['psf_bvec'][:15]/test4_gaussian_results[j][0]['psf_bvec'][0]
    
    plt.xlabel(r"pixel size (arcsec)")
    plt.ylabel(r'${\Delta g_i}$')
    
    plt.title(str((p,q)))
    #plt.show()
    plt.legend()
    
#fig.colorbar(axes)